In [2]:
import tensorflow as tf; print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices("GPU")))

2023-09-19 03:07:24.106469: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 03:07:24.817039: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cuda_nvrtc/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cudnn/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cublas/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cudnn/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/:/usr/local/cudnn-112-81133/lib64:/u

Num GPUs Available:  4


In [1]:
import sys
sys.path.append('./.local/lib/python3.8/site-packages')
sys.path.append('/usr/lib/python3/dist-packages')

from keras import losses, metrics
from keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization, Dropout, Activation, Input
from keras.layers import GlobalAveragePooling1D, Flatten, SeparableConv1D, concatenate, Add
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve
import tensorflow as tf
import itertools as it
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats, interp
import os, sys, pickle, shutil
import pandas as pd
import numpy as np
import random, datetime, time
from sklearn.model_selection import KFold
#from keras.utils.training_utils import multi_gpu_model


# tensorflow 사용 시 seed 고정
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
SEED = 98
GPU = 1
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]= f"{GPU}"

print(datetime.datetime.now())

2023-09-22 15:04:40.456455: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 15:04:41.226865: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cuda_nvrtc/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cudnn/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cublas/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/python3.8/site-packages/nvidia/cudnn/lib:/opt/anaconda-3-2020.02/envs/hskim/lib/:/usr/local/cudnn-112-81133/lib64:/u

2023-09-22 15:04:42.533924


# Input loading

## only signals

In [2]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'adult'
NORM = 'z-norm'
GENDER = 'female'

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{TRAIN}'
input_path = f'dataset/{hyper_path}/'

with np.load(input_path+'x.npz', allow_pickle=True) as f:
    x_train = f['x_train']
    x_test = f['x_test']
        
with np.load(input_path+'y.npz', allow_pickle=True) as f:
    y_train = f['y_train']
    y_test = f['y_test']
    
with np.load(input_path+'c.npz', allow_pickle=True) as f:
    c_train = f['c_train']
    c_test = f['c_test']  
    
with np.load(input_path+'g.npz', allow_pickle=True) as f:
    g_train = f['g_train']
    g_test = f['g_test']        
    
print(f'x train shape: {x_train.shape}')
print(f'x test shape: {x_test.shape}')

x train shape: (27844, 5000, 12)
x test shape: (6965, 5000, 12)


In [3]:
y_train = y_train / 100
y_test = y_test / 100

In [4]:
hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{GENDER}_{TRAIN}'

train_mask = (g_train == 1) if GENDER == 'male' else (g_train == 0)
test_mask = (g_test == 1) if GENDER == 'male' else (g_test == 0)

x_train = x_train[train_mask]
y_train = y_train[train_mask]
c_train = c_train[train_mask]
g_train = g_train[train_mask]

x_test = x_test[test_mask]
y_test = y_test[test_mask]
c_test = c_test[test_mask]
g_test = g_test[test_mask]

print(f'x train shape: {x_train.shape}')
print(f'y train shape: {y_train.shape}')
print(f'x test shape: {x_test.shape}')

x train shape: (14466, 5000, 12)
y train shape: (14466,)
x test shape: (3575, 5000, 12)


## signal + feats

### gender

In [ ]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'adult'
NORM = 'z-norm'
GENDER = 'female'

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{TRAIN}'
input_path = f'dataset/{hyper_path}/signal+features/'

with np.load(input_path+'x.npz', allow_pickle=True) as f:
    x_train = f['x_train']
    x_test = f['x_test']
     
with np.load(input_path+'feats.npz', allow_pickle=True) as f:
    feat_train = f['feat_train']
    feat_test = f['feat_test']
        
with np.load(input_path+'y.npz', allow_pickle=True) as f:
    y_train = f['y_train']
    y_test = f['y_test']
    
with np.load(input_path+'c.npz', allow_pickle=True) as f:
    c_train = f['c_train']
    c_test = f['c_test']  
    
with np.load(input_path+'g.npz', allow_pickle=True) as f:
    g_train = f['g_train']
    g_test = f['g_test']        
    
print(f'x train shape: {x_train.shape}')
print(f'feat train shape: {feat_train.shape}')
print(f'x test shape: {x_test.shape}')
print(f'feat test shape: {feat_test.shape}')

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{GENDER}_{TRAIN}'

train_mask = (g_train == 1) if GENDER == 'male' else (g_train == 0)
test_mask = (g_test == 1) if GENDER == 'male' else (g_test == 0)

x_train = x_train[train_mask]
feat_train = feat_train[train_mask]
y_train = y_train[train_mask]
c_train = c_train[train_mask]
g_train = g_train[train_mask]

x_test = x_test[test_mask]
feat_test = feat_test[test_mask]
y_test = y_test[test_mask]
c_test = c_test[test_mask]
g_test = g_test[test_mask]

print(f'x train shape: {x_train.shape}')
print(f'feat train shape: {feat_train.shape}')
print(f'x test shape: {x_test.shape}')
print(f'feat test shape: {feat_test.shape}')


SCALE_Y = 100 if ADULT == 'adult' else 20
y_train = y_train / SCALE_Y
y_test = y_test / SCALE_Y

print(f'train: {np.mean(y_train)}, test: {np.mean(y_test)}')

### male + female

In [2]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'adult'
NORM = 'z-norm'

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{TRAIN}'
input_path = f'dataset/{hyper_path}/signal+features/'

with np.load(input_path+'x.npz', allow_pickle=True) as f:
    x_train = f['x_train']
    x_test = f['x_test']
     
with np.load(input_path+'feats.npz', allow_pickle=True) as f:
    feat_train = f['feat_train']
    feat_test = f['feat_test']
        
with np.load(input_path+'y.npz', allow_pickle=True) as f:
    y_train = f['y_train']
    y_test = f['y_test']
    
with np.load(input_path+'c.npz', allow_pickle=True) as f:
    c_train = f['c_train']
    c_test = f['c_test']  
    
with np.load(input_path+'g.npz', allow_pickle=True) as f:
    g_train = f['g_train']
    g_test = f['g_test']        
    
print(f'x train shape: {x_train.shape}')
print(f'feat train shape: {feat_train.shape}')
print(f'x test shape: {x_test.shape}')
print(f'feat test shape: {feat_test.shape}')

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{TRAIN}'


SCALE_Y = 100
y_train = y_train / SCALE_Y
y_test = y_test / SCALE_Y

print(f'train: {np.mean(y_train)}, test: {np.mean(y_test)}')

x train shape: (27824, 5000, 12)
feat train shape: (27824, 55)
x test shape: (6962, 5000, 12)
feat test shape: (6962, 55)
train: 0.6083467602729797, test: 0.6087216138839722


### male, female

In [2]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'adult'
NORM = 'z-norm'

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{TRAIN}'
input_path = f'dataset/{hyper_path}/signal+features/'

with np.load(input_path+'x.npz', allow_pickle=True) as f:
    x_train = f['x_train']
    x_test = f['x_test']
     
with np.load(input_path+'feats.npz', allow_pickle=True) as f:
    feat_train = f['feat_train']
    feat_test = f['feat_test']
        
with np.load(input_path+'y.npz', allow_pickle=True) as f:
    y_train = f['y_train']
    y_test = f['y_test']
    
with np.load(input_path+'c.npz', allow_pickle=True) as f:
    c_train = f['c_train']
    c_test = f['c_test']  
    
with np.load(input_path+'g.npz', allow_pickle=True) as f:
    g_train = f['g_train']
    g_test = f['g_test']        
    
print(f'x train shape: {x_train.shape}')
print(f'feat train shape: {feat_train.shape}')
print(f'x test shape: {x_test.shape}')
print(f'feat test shape: {feat_test.shape}')


hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_M+F_{TRAIN}'

train_mask = (g_train == 1) 
test_mask = (g_test == 1)

x_train_m = x_train[train_mask]
feat_train_m = feat_train[train_mask]
y_train_m = y_train[train_mask]
c_train_m = c_train[train_mask]
g_train_m = g_train[train_mask]

x_test_m = x_test[test_mask]
feat_test_m = feat_test[test_mask]
y_test_m = y_test[test_mask]
c_test_m = c_test[test_mask]
g_test_m = g_test[test_mask]

print(f'x train shape: {x_train_m.shape}')
print(f'feat train shape: {feat_train_m.shape}')
print(f'x test shape: {x_test_m.shape}')
print(f'feat test shape: {feat_test_m.shape}')


SCALE_Y = 100
y_train_m = y_train_m / SCALE_Y
y_test_m = y_test_m / SCALE_Y

print(f'train: {np.mean(y_train_m)}, test: {np.mean(y_test_m)}')


train_fask = (g_train == 0) 
test_fask = (g_test == 0)

x_train_f = x_train[train_fask]
feat_train_f = feat_train[train_fask]
y_train_f = y_train[train_fask]
c_train_f = c_train[train_fask]
g_train_f = g_train[train_fask]

x_test_f = x_test[test_fask]
feat_test_f = feat_test[test_fask]
y_test_f = y_test[test_fask]
c_test_f = c_test[test_fask]
g_test_f = g_test[test_fask]

print(f'x train shape: {x_train_f.shape}')
print(f'feat train shape: {feat_train_f.shape}')
print(f'x test shape: {x_test_f.shape}')
print(f'feat test shape: {feat_test_f.shape}')


SCALE_Y = 100
y_train_f = y_train_f / SCALE_Y
y_test_f = y_test_f / SCALE_Y

print(f'train: {np.mean(y_train_f)}, test: {np.mean(y_test_f)}')

x train shape: (27824, 5000, 12)
feat train shape: (27824, 55)
x test shape: (6962, 5000, 12)
feat test shape: (6962, 55)
x train shape: (13358, 5000, 12)
feat train shape: (13358, 55)
x test shape: (3387, 5000, 12)
feat test shape: (3387, 55)
train: 0.6191248297691345, test: 0.6184705495834351
x train shape: (14466, 5000, 12)
feat train shape: (14466, 55)
x test shape: (3575, 5000, 12)
feat test shape: (3575, 55)
train: 0.5983941555023193, test: 0.5994852781295776


## signal + feats + aug

In [2]:
import numpy as np
from tqdm import tqdm

class lead_extractor:
    """
    used to select specific leads or random choice of configurations

    Twelve leads: I, II, III, V1, V2, V3, V4, V5, V6, aVR, aVL, aVF, 
    Six leads: I, II, V1, V3, V4, V6
    Four leads: III, V1, V2, aVL # I, II, III, V2
    Three leads: I, II, V2
    Two leads: I, II

    """
    L2 = np.array([1,1,0,0,0,0,0,0,0,0,0,0])
    L3 = np.array([1,1,0,0,0,0,0,1,0,0,0,0])
    L4 = np.array([0,0,1,1,1,0,0,0,0,0,1,0]) #([1,1,1,0,0,0,0,1,0,0,0,0])
    L6 = np.array([1,1,0,1,0,1,1,0,1,0,0,0])
    L8 = np.array([1,1,0,1,0,1,1,1,1,0,1,0])
    L12 = np.array([1,1,1,1,1,1,1,1,1,1,1,1])

    @staticmethod
    def get (x,num_leads):
        if num_leads==None:
            # random choice output
            num_leads = random.choice([12,8,6,4,3,2])

        if num_leads==12:
            # Twelve leads: I, II, III, aVR, aVL, aVF, V1, V2, V3, V4, V5, V6
            return x

        if num_leads==8:
            # Six leads: I, II, III, aVL, aVR, aVF
            x = x * lead_extractor.L8
            return x

        if num_leads==6:
            # Six leads: I, II, III, aVL, aVR, aVF
            x = x * lead_extractor.L6
            return x

        if num_leads==4:
            # Six leads: I, II, III, V2
            x = x * lead_extractor.L4
            return x

        if num_leads==3:
            # Three leads: I, II, V2
            x = x * lead_extractor.L3
            return x

        if num_leads==2:
            # Two leads: II, V5
            x = x * lead_extractor.L2
            return x
        raise Exception("invalid-leads-number")

def aug_generator(x_train, feat_train, y_train, c_train, g_train):
    from tqdm import tqdm
    x_train_, feat_train_, y_train_, c_train_, g_train_ = [], [], [], [], []
    for i in tqdm(range(x_train.shape[0])):
        x_train_.append(x_train[i])
        feat_train_.append(feat_train[i])
        y_train_.append(y_train[i])
        c_train_.append(c_train[i])
        g_train_.append(g_train[i])

        x_train_.append(lead_extractor.get(x_train[i], 8))
        feat_train_.append(feat_train[i])
        y_train_.append(y_train[i])
        c_train_.append(c_train[i])
        g_train_.append(g_train[i])    

        x_train_.append(lead_extractor.get(x_train[i], 6))
        feat_train_.append(feat_train[i])
        y_train_.append(y_train[i])
        c_train_.append(c_train[i])
        g_train_.append(g_train[i])  

        x_train_.append(lead_extractor.get(x_train[i], 4))
        feat_train_.append(feat_train[i])
        y_train_.append(y_train[i])
        c_train_.append(c_train[i])
        g_train_.append(g_train[i]) 

    x_train_ = np.array(x_train_, np.float32)
    feat_train_ = np.array(feat_train_, np.float32)
    y_train_ = np.array(y_train_, np.float32)
    c_train_ = np.array(c_train_)
    g_train_ = np.array(g_train_, np.float32)

    print(f'x shape: {x_train_.shape}')
    print(f'feat shape: {feat_train_.shape}')
    print(f'y shape: {y_train_.shape}')
    
    return x_train_, feat_train_, y_train_, c_train_, g_train_

In [3]:
SRATE = 500
ECG_FILT = 'bandpass'
TRAIN = 'train'
ADULT = 'adult'
NORM = 'z-norm'
GENDER = 'male'

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{TRAIN}'
input_path = f'dataset/{hyper_path}/signal+features/'

with np.load(input_path+'x.npz', allow_pickle=True) as f:
    x_train = f['x_train']
    x_test = f['x_test']
     
with np.load(input_path+'feats.npz', allow_pickle=True) as f:
    feat_train = f['feat_train']
    feat_test = f['feat_test']
        
with np.load(input_path+'y.npz', allow_pickle=True) as f:
    y_train = f['y_train']
    y_test = f['y_test']
    
with np.load(input_path+'c.npz', allow_pickle=True) as f:
    c_train = f['c_train']
    c_test = f['c_test']  
    
with np.load(input_path+'g.npz', allow_pickle=True) as f:
    g_train = f['g_train']
    g_test = f['g_test']        
    
print(f'x train shape: {x_train.shape}')
print(f'feat train shape: {feat_train.shape}')
print(f'x test shape: {x_test.shape}')
print(f'feat test shape: {feat_test.shape}')

hyper_path = f'SRATE{SRATE}_ECG-{ECG_FILT}_{NORM}_{ADULT}_{GENDER}_{TRAIN}'

train_mask = (g_train == 1) if GENDER == 'male' else (g_train == 0)
test_mask = (g_test == 1) if GENDER == 'male' else (g_test == 0)

x_train = x_train[train_mask]
feat_train = feat_train[train_mask]
y_train = y_train[train_mask]
c_train = c_train[train_mask]
g_train = g_train[train_mask]

x_test = x_test[test_mask]
feat_test = feat_test[test_mask]
y_test = y_test[test_mask]
c_test = c_test[test_mask]
g_test = g_test[test_mask]

print(f'x train shape: {x_train.shape}')
print(f'feat train shape: {feat_train.shape}')
print(f'y train shape: {y_train.shape}')
print(f'x test shape: {x_test.shape}')
print(f'feat test shape: {feat_test.shape}')


# augmentation
x_train, feat_train, y_train, c_train, g_train = aug_generator(x_train, feat_train, y_train, c_train, g_train)
#x_test, feat_test, y_test, c_test, g_test = aug_generator(x_test, feat_test, y_test, c_test, g_test)

SCALE_Y = 100
y_train = y_train / SCALE_Y
y_test = y_test / SCALE_Y

print(f'train: {np.mean(y_train)}, test: {np.mean(y_test)}')

x train shape: (27824, 5000, 12)
feat train shape: (27824, 55)
x test shape: (6962, 5000, 12)
feat test shape: (6962, 55)
x train shape: (13358, 5000, 12)
feat train shape: (13358, 55)
y train shape: (13358,)
x test shape: (3387, 5000, 12)
feat test shape: (3387, 55)


100%|██████████| 13358/13358 [00:09<00:00, 1342.08it/s]


x shape: (53432, 5000, 12)
feat shape: (53432, 55)
y shape: (53432,)
train: 0.6191248893737793, test: 0.6184705495834351


# Settings

In [6]:
# folder
nfold = 4  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 500
rootdir = f"randomSearch/{hyper_path}/CNN_4layers_age%100_{nfold}fold_test{ntest}"

if not os.path.exists(f"randomSearch/{hyper_path}"):
    os.mkdir(f"randomSearch/{hyper_path}")

if not os.path.exists(rootdir):
    os.mkdir(rootdir)

# 모델에 대한 정보 txt로 저장
f = open(f'{rootdir}/README.txt', 'w')
f.write(f'model: 1D CNN 4 layers, regression')
f.write(f'input: ECG of 10 second, output: age')
f.close()
    

# test_settings
layer_settings, test_settings = [], []


# hyperparamters
globalpool_opts = ['max','ave']

# hyperparamters pool
filt_opts = [16, 32, 64, 128] # num of filters(kernel)
stride_opts = [1,2,3]  # other opts: stride = (kernel-1)/2
kernel_opts = range(3,11,2) # kernel size
pool_size = 2
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [0, 8, 16, 32, 64]
BATCH_SIZE = [32, 64, 128, 256, 512]
lr_opts = [0.001, 0.002, 0.0005]

print('start making test settings...', end='', flush=True)
# test settings
nfilt, kernels, strides = [], [], []
for i in range(5):
    nfilt.append(0)
    kernels.append(0)
    strides.append(0)

for nfilter in filt_opts:
    for kernel in kernel_opts:
        for stride in stride_opts:
        #layer_settings.append([nfilter, kernel, int((kernel-1)/2)])       
            layer_settings.append([nfilter, kernel, stride])
    
for dense_node in dense_opts:
    for dropout_cnn in dropout_opts:
        for dropout_fc in dropout_opts:
            for batch_size in BATCH_SIZE:
                for learning_rate in lr_opts:
                    test_settings.append([dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate])                                   

                        
print('done')
print(datetime.datetime.now())

start making test settings...done
2023-09-21 02:01:44.711440


## 1D-CNN

In [ ]:
from keras import metrics
from tensorflow.keras.callbacks import LearningRateScheduler

seed_everything(SEED)

# random search for hyperparameter
ntrial = ntest
train_errs, val_errs = [] ,[]
test_acc, test_roc, test_prc = [], [], []
#test_rmse, test_mae, test_auc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    
    # total conv layers of the model
    n_conv = random.choice([2,3,4]) 
    # test settings
    for i in range(n_conv):
        nfilt[i], kernels[i], strides[i] = random.choice(layer_settings)
    dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate = random.choice(test_settings)
    
    if itrial < 0:
        continue
    

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = f'batch{batch_size},'
    for i in range(n_conv):
        odir_f += f'layer{i+1}:c{nfilt[i]}filt{kernels[i]}str{strides[i]},'
    odir_f += f'1conv,dropout{dropout_cnn},dnodes{dense_node},dropout{dropout_fc}'#,lr{learning_rate}'
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

        
    # model validation (VGG)
    out_shape = x_train.shape[1]
    for i in range(n_conv):
        out_shape = out_shape / pool_size / strides[i]
    if out_shape < 1:
        print('non-valid structure')
        os.rmdir(odir)
        itrial -= 1
        continue
        
    #mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/GPU:0","/GPU:1","/GPU:2","/GPU:3"])
    #with mirrored_strategy.scope(): 
    
    with tf.device(f'/gpu:{GPU}'):
        # build a model
        inp = Input(shape=(x_train.shape[1],x_train.shape[2]))
        out = inp


        # VGC block
        for i in range(n_conv):
            out = Conv1D(filters=nfilt[i], kernel_size=kernels[i], strides=strides[i], padding='same', activation='relu')(out)
            out = BatchNormalization()(out)
            out = MaxPooling1D(pool_size)(out)


        # globalpooling vs flattening vs 1x1 convolution
        #elif globalpool_opt == 'ave':
        #    out = GlobalAveragePooling1D()(out)
        out = Conv1D(filters=1, kernel_size=1)(out)
        out = Flatten() (out)



        if dense_node != 0:
            out = Dropout(dropout_cnn)(out)
            out = Dense(dense_node, activation='relu')(out)
        out = Dropout(dropout_fc)(out)
        out = Dense(1, activation='linear')(out)


        model = Model(inputs=[inp], outputs=[out])
        model.save_weights(f'{odir}/initial_weights.hdf5')


        # 4-fold cv
        kfold = KFold(nfold)
        tprs, aucs, prs = [], [], []
        test_rmse, test_mae = [], []
        f1_scores, thvals = [], []
        mean_fpr = np.linspace(0,1,100)
        mean_recall = np.linspace(0,1,100)

        switch = 0
        caseids_train = np.unique(c_train)
        for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
            c_cv_trains = caseids_train[c_cv_trains_mask]
            
            cv_train_mask = np.isin(c_train, c_cv_trains)
            cv_val_mask = ~cv_train_mask
            
            X_train = x_train[cv_train_mask]
            X_val = x_train[cv_val_mask]

            Y_train = y_train[cv_train_mask]
            Y_val = y_train[cv_val_mask]


            # model 학습
            try:
                # learning scheduler
                def step_decay(epoch):
                    start = 1e-3
                    drop = 0.1
                    epochs_drop = 10
                    lr = start * (drop ** np.floor((epoch)/epochs_drop))
                    return lr
                lr_scheduler = LearningRateScheduler(step_decay, verbose=1)
                
                weightcache = f"{odir}/weights_{fold}.hdf5"
                #model = multi_gpu_model(model, gpus=4)
                model.compile(loss='mean_squared_error', optimizer=Adam(lr=lr_scheduler, weight_decay=1e-4), metrics=['mean_absolute_error'])
                hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                        callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                    EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

                model.load_weights(weightcache)
                y_pred = model.predict(x_test).flatten()

                # MAE 계산
                model_err = metrics.MeanAbsoluteError()
                model_err.update_state(y_test, y_pred)
                mae_val = model_err.result().numpy()
                test_mae.append(mae_val)

                # RMSE 계산
                model_err = metrics.RootMeanSquaredError() 
                model_err.update_state(y_test, y_pred)
                rmse_val = model_err.result().numpy()
                test_rmse.append(rmse_val)


                print(f' ###{fold} fold : val mae {mae_val:.2f}, val rmse {rmse_val:.2f}###')
                tf.keras.backend.clear_session()
                model.load_weights(f'{odir}/initial_weights.hdf5')

            except Exception as e:
                print(e)
                switch = 1
                shutil.rmtree(odir)
                itrial -= 1
                break
        ###
        if switch:
            switch = 0
            tf.keras.backend.clear_session()
            continue

    # 

    # RMSE 계산
    mean_rmse = np.mean(test_rmse)
    std_rmse = np.std(test_rmse)
    
    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))
    

    print(f'mae{mean_mae*100:.2f}+-{std_mae*100:.2f}_rmse{mean_rmse*100:.2f}+-{std_rmse*100:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*100:.2f}+-{std_mae*100:.2f}_rmse{mean_rmse*100:.2f}+-{std_rmse*100:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


random search 0/500


Epoch 1/100
340/340 [==============================] - ETA: 0s - loss: 0.1701 - mean_absolute_error: 0.2734
Epoch 1: val_loss improved from inf to 0.03184, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN_4layers_age%100_4fold_test500/batch32,layer1:c128filt3str1,layer2:c16filt3str3,layer3:c32filt9str2,1conv,dropout0.1,dnodes32,dropout0/weights_0.hdf5
340/340 [==============================] - 10s 21ms/step - loss: 0.1701 - mean_absolute_error: 0.2734 - val_loss: 0.0318 - val_mean_absolute_error: 0.1442
Epoch 2/100
340/340 [==============================] - ETA: 0s - loss: 0.0276 - mean_absolute_error: 0.1311
Epoch 2: val_loss improved from 0.03184 to 0.02194, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN_4layers_age%100_4fold_test500/batch32,layer1:c128filt3str1,layer2:c16filt3str3,layer3:c32filt9str2,1conv,dropout0.1,dnodes32,dropout0/weights_0.hdf5
340/340 [==============================] - 6s 19ms/step - loss: 0.027

Epoch 1/100
337/340 [============================>.] - ETA: 0s - loss: 0.1604 - mean_absolute_error: 0.2643
Epoch 1: val_loss improved from inf to 0.02790, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN_4layers_age%100_4fold_test500/batch32,layer1:c128filt3str1,layer2:c16filt3str3,layer3:c32filt9str2,1conv,dropout0.1,dnodes32,dropout0/weights_1.hdf5
340/340 [==============================] - 10s 22ms/step - loss: 0.1596 - mean_absolute_error: 0.2635 - val_loss: 0.0279 - val_mean_absolute_error: 0.1349
Epoch 2/100
338/340 [============================>.] - ETA: 0s - loss: 0.0274 - mean_absolute_error: 0.1312
Epoch 2: val_loss improved from 0.02790 to 0.01988, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN_4layers_age%100_4fold_test500/batch32,layer1:c128filt3str1,layer2:c16filt3str3,layer3:c32filt9str2,1conv,dropout0.1,dnodes32,dropout0/weights_1.hdf5
340/340 [==============================] - 6s 19ms/step - loss: 0.027

Epoch 1/100
340/340 [==============================] - ETA: 0s - loss: 0.1519 - mean_absolute_error: 0.2618
Epoch 1: val_loss improved from inf to 0.04487, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN_4layers_age%100_4fold_test500/batch32,layer1:c128filt3str1,layer2:c16filt3str3,layer3:c32filt9str2,1conv,dropout0.1,dnodes32,dropout0/weights_2.hdf5
340/340 [==============================] - 10s 22ms/step - loss: 0.1519 - mean_absolute_error: 0.2618 - val_loss: 0.0449 - val_mean_absolute_error: 0.1739
Epoch 2/100
338/340 [============================>.] - ETA: 0s - loss: 0.0269 - mean_absolute_error: 0.1305
Epoch 2: val_loss improved from 0.04487 to 0.02119, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN_4layers_age%100_4fold_test500/batch32,layer1:c128filt3str1,layer2:c16filt3str3,layer3:c32filt9str2,1conv,dropout0.1,dnodes32,dropout0/weights_2.hdf5
340/340 [==============================] - 6s 19ms/step - loss: 0.026

Epoch 1/100
338/340 [============================>.] - ETA: 0s - loss: 0.1548 - mean_absolute_error: 0.2621
Epoch 1: val_loss improved from inf to 0.02998, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN_4layers_age%100_4fold_test500/batch32,layer1:c128filt3str1,layer2:c16filt3str3,layer3:c32filt9str2,1conv,dropout0.1,dnodes32,dropout0/weights_3.hdf5
340/340 [==============================] - 10s 22ms/step - loss: 0.1545 - mean_absolute_error: 0.2617 - val_loss: 0.0300 - val_mean_absolute_error: 0.1382
Epoch 2/100
340/340 [==============================] - ETA: 0s - loss: 0.0274 - mean_absolute_error: 0.1309
Epoch 2: val_loss improved from 0.02998 to 0.02250, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN_4layers_age%100_4fold_test500/batch32,layer1:c128filt3str1,layer2:c16filt3str3,layer3:c32filt9str2,1conv,dropout0.1,dnodes32,dropout0/weights_3.hdf5
340/340 [==============================] - 6s 19ms/step - loss: 0.027

Epoch 1/100
84/85 [============================>.] - ETA: 0s - loss: 1.2236 - mean_absolute_error: 0.8060
Epoch 1: val_loss improved from inf to 0.22812, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN_4layers_age%100_4fold_test500/batch128,layer1:c16filt5str2,layer2:c64filt7str2,layer3:c32filt3str1,layer4:c64filt3str2,1conv,dropout0.1,dnodes0,dropout0.3/weights_0.hdf5
85/85 [==============================] - 6s 33ms/step - loss: 1.2166 - mean_absolute_error: 0.8034 - val_loss: 0.2281 - val_mean_absolute_error: 0.4024
Epoch 2/100
85/85 [==============================] - ETA: 0s - loss: 0.2186 - mean_absolute_error: 0.3712
Epoch 2: val_loss improved from 0.22812 to 0.07423, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN_4layers_age%100_4fold_test500/batch128,layer1:c16filt5str2,layer2:c64filt7str2,layer3:c32filt3str1,layer4:c64filt3str2,1conv,dropout0.1,dnodes0,dropout0.3/weights_0.hdf5
85/85 [===========================

Epoch 1/100
85/85 [==============================] - ETA: 0s - loss: 1.2073 - mean_absolute_error: 0.8082
Epoch 1: val_loss improved from inf to 0.25232, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN_4layers_age%100_4fold_test500/batch128,layer1:c16filt5str2,layer2:c64filt7str2,layer3:c32filt3str1,layer4:c64filt3str2,1conv,dropout0.1,dnodes0,dropout0.3/weights_1.hdf5
85/85 [==============================] - 7s 39ms/step - loss: 1.2073 - mean_absolute_error: 0.8082 - val_loss: 0.2523 - val_mean_absolute_error: 0.4378
Epoch 2/100
85/85 [==============================] - ETA: 0s - loss: 0.2387 - mean_absolute_error: 0.3909
Epoch 2: val_loss improved from 0.25232 to 0.07901, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN_4layers_age%100_4fold_test500/batch128,layer1:c16filt5str2,layer2:c64filt7str2,layer3:c32filt3str1,layer4:c64filt3str2,1conv,dropout0.1,dnodes0,dropout0.3/weights_1.hdf5
85/85 [===========================

In [10]:
inp = Input(shape=(x_train.shape[1],x_train.shape[2]))
out = inp

# dropout for lead channels
print(out.shape)
out = Dropout(0.2, noise_shape=(batch_size, 1, 12))(out)
print(out.shape)

(None, 5000, 12)
(None, 5000, 12)


## 1d-cnn + features

In [3]:
# folder
nfold = 4  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 500
rootdir = f"randomSearch/{hyper_path}/CNN_4layers+feats_kernel(7~19)_age%{SCALE_Y}(sigmoid)_mse(decay)_{nfold}fold_test{ntest}"

if not os.path.exists(f"randomSearch/{hyper_path}"):
    os.mkdir(f"randomSearch/{hyper_path}")

if not os.path.exists(rootdir):
    os.mkdir(rootdir)

# 모델에 대한 정보 txt로 저장
f = open(f'{rootdir}/README.txt', 'w')
f.write(f'model: 1D CNN 4 layers, regression')
f.write(f'input: ECG of 10 second, output: age')
f.close()
    

# test_settings
layer_settings, test_settings = [], []


# hyperparamters
globalpool_opts = ['max','ave']

# hyperparamters pool
filt_opts = [16, 32, 64] # num of filters(kernel)
stride_opts = [2,4]  # other opts: stride = (kernel-1)/2
kernel_opts = range(7,20,2) # kernel size
pool_size = 2
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [0, 16, 32, 64, 128]
BATCH_SIZE = [32, 64, 128, 256]
lr_opts = [0.001, 0.002, 0.0005]

print('start making test settings...', end='', flush=True)
# test settings
nfilt, kernels, strides = [], [], []
for i in range(5):
    nfilt.append(0)
    kernels.append(0)
    strides.append(0)

for nfilter in filt_opts:
    for kernel in kernel_opts:
        for stride in stride_opts:
        #layer_settings.append([nfilter, kernel, int((kernel-1)/2)])       
            layer_settings.append([nfilter, kernel, stride])
    
for dense_node in dense_opts:
    for dropout_cnn in dropout_opts:
        for dropout_fc in dropout_opts:
            for batch_size in BATCH_SIZE:
                for learning_rate in lr_opts:
                    test_settings.append([dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate])                                   

                        
print('done')
print(datetime.datetime.now())

start making test settings...done
2023-09-22 11:16:06.138500


In [ ]:
from keras import metrics
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.layers import Concatenate
from sklearn.metrics import mean_absolute_error, mean_squared_error

seed_everything(SEED)

def step_decay(epoch):
    start = 1e-3
    drop = 0.1
    epochs_drop = 10
    lr = start * (drop ** np.floor((epoch)/epochs_drop))
    return lr

# random search for hyperparameter
ntrial = ntest
train_errs, val_errs = [] ,[]
test_acc, test_roc, test_prc = [], [], []
#test_rmse, test_mae, test_auc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    
    # total conv layers of the model
    n_conv = random.choice([2,3,4]) 
    # test settings
    for i in range(n_conv):
        nfilt[i], kernels[i], strides[i] = random.choice(layer_settings)
    dense_node, dropout_cnn, dropout_fc, batch_size, learning_rate = random.choice(test_settings)
    
    if itrial < 0:
        continue
    

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = f'batch{batch_size},'
    for i in range(n_conv):
        odir_f += f'layer{i+1}:c{nfilt[i]}filt{kernels[i]}str{strides[i]},'
    odir_f += f'1conv,dropout{dropout_cnn},dnodes{dense_node},dropout{dropout_fc}'#,lr{learning_rate}'
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

        
    # model validation (VGG)
    out_shape = x_train.shape[1]
    for i in range(n_conv):
        out_shape = out_shape / pool_size / strides[i]
    if out_shape < 1:
        print('non-valid structure')
        os.rmdir(odir)
        itrial -= 1
        continue
        
    #mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/GPU:0","/GPU:1","/GPU:2","/GPU:3"])
    #with mirrored_strategy.scope(): 
    
    with tf.device(f'/gpu:{GPU}'):
        # build a model
        inp = Input(shape=(x_train.shape[1],x_train.shape[2]), name="input_1")
        out = inp


        # VGC block
        for i in range(n_conv):
            out = Conv1D(filters=nfilt[i], kernel_size=kernels[i], strides=strides[i], padding='same', activation=None)(out)
            out = BatchNormalization()(out)
            out = Activation('relu')(out)
            out = MaxPooling1D(pool_size)(out)


        # globalpooling vs flattening vs 1x1 convolution
        #elif globalpool_opt == 'ave':
        #    out = GlobalAveragePooling1D()(out)
        out = Conv1D(filters=1, kernel_size=1)(out)
        out = Flatten() (out)

    
        # concat with X_feature
        inp_feat = Input(shape=(feat_train.shape[1]), name="input_2")
        out = Concatenate(axis=1)([out, inp_feat])

        if dense_node != 0:
            out = Dropout(dropout_cnn)(out)
            out = Dense(dense_node, activation='relu')(out)
        out = Dropout(dropout_fc)(out)
        out = Dense(1, activation='sigmoid')(out)


        model = Model(inputs=[inp, inp_feat], outputs=[out])
        lr_scheduler = LearningRateScheduler(step_decay, verbose=1)
        model.compile(loss='mean_squared_error', optimizer=Adam(lr=lr_scheduler, weight_decay=1e-4), metrics=['mean_absolute_error'])
        model.save_weights(f'{odir}/initial_weights.hdf5')


    # 4-fold cv
    kfold = KFold(nfold)
    tprs, aucs, prs = [], [], []
    test_rmse, test_mae = [], []
    f1_scores, thvals = [], []
    mean_fpr = np.linspace(0,1,100)
    mean_recall = np.linspace(0,1,100)

    switch = 0
    caseids_train = np.unique(c_train)
    for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
        c_cv_trains = caseids_train[c_cv_trains_mask]

        cv_train_mask = np.isin(c_train, c_cv_trains)
        cv_val_mask = ~cv_train_mask

        X_train = x_train[cv_train_mask]
        X_val = x_train[cv_val_mask]

        Y_train = y_train[cv_train_mask]
        Y_val = y_train[cv_val_mask]

        Feat_train = feat_train[cv_train_mask]
        Feat_val = feat_train[cv_val_mask]

        
        # model 학습
        try:

            with tf.device("/CPU:0"):
                train_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": X_train, "input_2": Feat_train}, Y_train)).shuffle(x_train.shape[0]).batch(batch_size)
                val_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": X_val, "input_2": Feat_val}, Y_val)).batch(batch_size)
                test_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": x_test, "input_2": feat_test}, y_test)).batch(batch_size)
       
            weightcache = f"{odir}/weights_{fold}.hdf5"
            #model = multi_gpu_model(model, gpus=4)
            #model.compile(loss='mean_squared_error', optimizer=Adam(lr=lr_scheduler, weight_decay=1e-4), metrics=['mean_absolute_error'])
            hist = model.fit(train_dataset, validation_data=val_dataset, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

            model.load_weights(weightcache)
            y_pred = model.predict(test_dataset).flatten()

            # MAE 계산
            mae_val = mean_absolute_error(y_test, y_pred)
            test_mae.append(mae_val)


            print(f' ###{fold} fold : val mae {mae_val:.2f}###')
            tf.keras.backend.clear_session()
            model.load_weights(f'{odir}/initial_weights.hdf5')

        except Exception as e:
            print(e)
            switch = 1
            shutil.rmtree(odir)
            itrial -= 1

            break
    ###
    if switch:
        switch = 0
        tf.keras.backend.clear_session()
        continue

    # 

    
    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))
    

    print(f'mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


random search 0/500


2023-09-22 11:16:09.301389: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 11:16:09.791837: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22324 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:82:00.0, compute capability: 8.6


Epoch 1/100


2023-09-22 11:16:20.641364: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2023-09-22 11:16:21.128042: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-22 11:16:21.213110: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-22 11:16:21.215772: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x1c9d3a20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-22 11:16:21.215819: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2023-09-22 11:16:21.224191: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-22 11:16:21.297833: I tensor

164/164 [==============================] - ETA: 0s - loss: 0.0395 - mean_absolute_error: 0.1534
Epoch 1: val_loss improved from inf to 0.01533, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch128,layer1:c64filt15str2,layer2:c16filt9str2,layer3:c32filt15str2,1conv,dropout0.5,dnodes64,dropout0.4/weights_0.hdf5
164/164 [==============================] - 12s 37ms/step - loss: 0.0395 - mean_absolute_error: 0.1534 - val_loss: 0.0153 - val_mean_absolute_error: 0.0958
Epoch 2/100
163/164 [============================>.] - ETA: 0s - loss: 0.0201 - mean_absolute_error: 0.1101
Epoch 2: val_loss did not improve from 0.01533
164/164 [==============================] - 5s 29ms/step - loss: 0.0201 - mean_absolute_error: 0.1101 - val_loss: 0.0156 - val_mean_absolute_error: 0.0967
Epoch 3/100
162/164 [============================>.] - ETA: 0s - loss: 0.0156 - mean_absolute_error: 0.0979
Epoch 3: val_loss 

...

Epoch 1/100
82/82 [==============================] - ETA: 0s - loss: 0.0247 - mean_absolute_error: 0.1210
Epoch 1: val_loss improved from inf to 0.01761, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch256,layer1:c64filt15str2,layer2:c32filt7str2,layer3:c16filt7str2,1conv,dropout0,dnodes64,dropout0.2/weights_0.hdf5
82/82 [==============================] - 8s 67ms/step - loss: 0.0247 - mean_absolute_error: 0.1210 - val_loss: 0.0176 - val_mean_absolute_error: 0.1048
Epoch 2/100
81/82 [============================>.] - ETA: 0s - loss: 0.0151 - mean_absolute_error: 0.0957
Epoch 2: val_loss improved from 0.01761 to 0.01439, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch256,layer1:c64filt15str2,layer2:c32filt7str2,layer3:c16filt7str2,1conv,dropout0,dnodes64,dropout0.2/weights_0.hdf5
82/82 [===

Epoch 1/100
82/82 [==============================] - ETA: 0s - loss: 0.0437 - mean_absolute_error: 0.1606
Epoch 1: val_loss improved from inf to 0.05838, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch256,layer1:c64filt17str2,layer2:c64filt9str4,1conv,dropout0.4,dnodes0,dropout0.3/weights_0.hdf5
82/82 [==============================] - 8s 69ms/step - loss: 0.0437 - mean_absolute_error: 0.1606 - val_loss: 0.0584 - val_mean_absolute_error: 0.2056
Epoch 2/100
81/82 [============================>.] - ETA: 0s - loss: 0.0210 - mean_absolute_error: 0.1132
Epoch 2: val_loss improved from 0.05838 to 0.01798, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch256,layer1:c64filt17str2,layer2:c64filt9str4,1conv,dropout0.4,dnodes0,dropout0.3/weights_0.hdf5
82/82 [==============================] - 4s 51ms

Epoch 1/100
163/164 [============================>.] - ETA: 0s - loss: 0.0181 - mean_absolute_error: 0.1025
Epoch 1: val_loss improved from inf to 0.01365, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch128,layer1:c32filt15str4,layer2:c64filt7str2,layer3:c32filt19str4,layer4:c16filt19str2,1conv,dropout0,dnodes128,dropout0/weights_0.hdf5
164/164 [==============================] - 9s 32ms/step - loss: 0.0181 - mean_absolute_error: 0.1025 - val_loss: 0.0137 - val_mean_absolute_error: 0.0914
Epoch 2/100
163/164 [============================>.] - ETA: 0s - loss: 0.0122 - mean_absolute_error: 0.0860
Epoch 2: val_loss improved from 0.01365 to 0.01220, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch128,layer1:c32filt15str4,layer2:c64filt7str2,layer3:c32filt19str4,layer4:c16filt19str2,1conv,dropo

Epoch 1/100
82/82 [==============================] - ETA: 0s - loss: 0.0470 - mean_absolute_error: 0.1670
Epoch 1: val_loss improved from inf to 0.02020, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch256,layer1:c16filt9str2,layer2:c64filt17str2,layer3:c32filt13str2,layer4:c32filt17str2,1conv,dropout0.2,dnodes64,dropout0.2/weights_0.hdf5
82/82 [==============================] - 9s 63ms/step - loss: 0.0470 - mean_absolute_error: 0.1670 - val_loss: 0.0202 - val_mean_absolute_error: 0.1099
Epoch 2/100
81/82 [============================>.] - ETA: 0s - loss: 0.0235 - mean_absolute_error: 0.1193
Epoch 2: val_loss did not improve from 0.02020
82/82 [==============================] - 3s 40ms/step - loss: 0.0234 - mean_absolute_error: 0.1192 - val_loss: 0.0204 - val_mean_absolute_error: 0.1106
Epoch 3/100
81/82 [============================>.] - ETA: 0s - loss: 0.0181 - mean_absolute_error: 0.1

Epoch 1/100
651/653 [============================>.] - ETA: 0s - loss: 0.0296 - mean_absolute_error: 0.1310
Epoch 1: val_loss improved from inf to 0.01696, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_train/CNN_4layers+feats_kernel(7~19)_age%100(sigmoid)_mse(decay)_4fold_test500/batch32,layer1:c32filt7str4,layer2:c64filt15str2,layer3:c64filt9str4,1conv,dropout0.2,dnodes0,dropout0.3/weights_0.hdf5
653/653 [==============================] - 13s 16ms/step - loss: 0.0296 - mean_absolute_error: 0.1309 - val_loss: 0.0170 - val_mean_absolute_error: 0.1048
Epoch 2/100
532/653 [=======================>......] - ETA: 1s - loss: 0.0138 - mean_absolute_error: 0.0925

### male + female (loss = loss_f + loss_m)

## CNN-transformer

In [ ]:
random.seed(98)

from keras.models import Model
from keras.layers import Layer, LayerNormalization, Dense, Dropout, Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Input, concatenate, multiply, dot, MultiHeadAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler


SRATE = 500  # in hz
SEGLEN = 10 * SRATE  # samples
#BATCH_SIZE = 256
MAX_CASES = 500
nfold = 4

hyperparameters = {
    "nfilt" : [16, 32, 64],
    'nhead' : [2, 4, 8],
    'kdim': [16, 64, 128, 256],
    "fnode" : [32, 64, 128],
    "clayer" : [2, 3, 4],
    "tlayer" : [3, 4, 5],
    "droprate" : [0.1, 0.2, 0.5],
    "filtsize" : [5, 7, 9, 11, 15, 19],
    'poolsize' : [2, 4, 5],
    'batch_size': [64, 128, 256]
}
keys, values = zip(*hyperparameters.items())
permutations_dicts = it.product(*values)
permutations_dicts = list(permutations_dicts)
random.shuffle(permutations_dicts)
for nfilt, nhead, kdim, fnode, clayer, tlayer, droprate, filtsize, poolsize, batch_size in permutations_dicts:
    test_start = time.time()

    tf.keras.backend.clear_session()
    
    rootdir = f'randomSearch/{hyper_path}/CNN+transformer_age%100(sigmoid)_loss(mse)_{nfold}fold'
    odir_f = 'batch{}_cnn{}_filt{}_size{}_pool{}_do{}'.format(batch_size, clayer, nfilt, filtsize, poolsize, droprate)
    odir_f += '_tra{}_head{}_kdim{}_fnn{}'.format(tlayer, nhead, kdim, fnode)
    
    if not os.path.exists(rootdir):
        os.mkdir(rootdir)
    
    odir = rootdir+'/'+odir_f
    print("============================")
    print(odir)
    print("============================")

    # cnn-transformer
    # https://keras.io/examples/nlp/text_classification_with_transformer/
    out = inp = Input(shape=(x_train.shape[1], x_train.shape[2]))
    
    # out = Conv1D(filters=kdim, kernel_size=filtsize, padding='same')(out)

    # conv 여러층    
    for i in range(clayer):
        out = Conv1D(filters=nfilt, kernel_size=filtsize, padding='same', activation=None)(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
        out = MaxPooling1D(poolsize, padding='same')(out)
    out = Dense(kdim)(out)  # 마지막 차원이 nfilt 인데 kdim 으로 바꿔야 transformer block을 쌓을 수 있다.
    for i in range(tlayer):  # transformer
        attn_output = MultiHeadAttention(num_heads=nhead, key_dim=kdim, attention_axes=[1,])(out, out)
        attn_output = Dropout(droprate)(attn_output)
        out1 = LayerNormalization(epsilon=1e-6)(out + attn_output)  # sum and norm
        ffn_output = tf.keras.Sequential([Dense(fnode, activation="relu"), Dense(kdim)])(out1)
        out2 = Dropout(droprate)(ffn_output)
        out = LayerNormalization(epsilon=1e-6)(out1 + out2)  # sum and norm
    out = GlobalMaxPooling1D()(out)

    if droprate:
        out = Dropout(droprate)(out)
    out = Dense(fnode)(out)
    if droprate:
        out = Dropout(droprate)(out)
    out = Dense(1, activation='sigmoid')(out)

    if not os.path.exists(odir):
        os.mkdir(odir)

    #cache_path = odir + "/initial_weights.hdf5"
    model = Model(inputs=[inp], outputs=[out])
    model.save_weights(f'{odir}/initial_weights.hdf5')
    
    
    # 4-fold cv
    kfold = KFold(nfold)
    tprs, aucs, prs = [], [], []
    test_rmse, test_mae = [], []
    f1_scores, thvals = [], []
    mean_fpr = np.linspace(0,1,100)
    mean_recall = np.linspace(0,1,100)

    switch = 0
    caseids_train = np.unique(c_train)
    for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
        c_cv_trains = caseids_train[c_cv_trains_mask]

        cv_train_mask = np.isin(c_train, c_cv_trains)
        cv_val_mask = ~cv_train_mask

        X_train = x_train[cv_train_mask]
        X_val = x_train[cv_val_mask]

        Y_train = y_train[cv_train_mask]
        Y_val = y_train[cv_val_mask]


        # model 학습
        try:
            # learning scheduler
            def step_decay(epoch):
                start = 1e-3
                drop = 0.1
                epochs_drop = 10
                lr = start * (drop ** np.floor((epoch)/epochs_drop))
                return lr
            lr_scheduler = LearningRateScheduler(step_decay, verbose=1)

            weightcache = f"{odir}/weights_{fold}.hdf5"
            #model = multi_gpu_model(model, gpus=4)
            model.compile(loss='mse', optimizer=Adam(lr=lr_scheduler, weight_decay=1e-4), metrics=['mean_absolute_error'])
            hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

            model.load_weights(weightcache)
            y_pred = model.predict(x_test).flatten()

            # MAE 계산
            model_err = metrics.MeanAbsoluteError()
            model_err.update_state(y_test, y_pred)
            mae_val = model_err.result().numpy()
            test_mae.append(mae_val)

            # RMSE 계산
            model_err = metrics.RootMeanSquaredError() 
            model_err.update_state(y_test, y_pred)
            rmse_val = model_err.result().numpy()
            test_rmse.append(rmse_val)


            print(f' ###{fold} fold : val mae {mae_val:.2f}, val rmse {rmse_val:.2f}###')
            tf.keras.backend.clear_session()
            model.load_weights(f'{odir}/initial_weights.hdf5')

        except Exception as e:
            print(e)
            switch = 1
            shutil.rmtree(odir)
            break
    ###
    if switch:
        switch = 0
        tf.keras.backend.clear_session()
        continue


    # RMSE 계산
    mean_rmse = np.mean(test_rmse)
    std_rmse = np.std(test_rmse)

    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))


    print(f'mae{mean_mae*100:.2f}+-{std_mae*100:.2f}_rmse{mean_rmse*100:.2f}+-{std_rmse*100:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*100:.2f}+-{std_mae*100:.2f}_rmse{mean_rmse*100:.2f}+-{std_rmse*100:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN+transformer_age%100(sigmoid)_loss(mse)_4fold/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128


2023-09-21 11:04:26.367581: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 11:04:26.848920: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22324 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:82:00.0, compute capability: 8.6


Epoch 1/100


2023-09-21 11:04:36.912737: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2023-09-21 11:04:37.966387: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-21 11:04:38.041882: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-21 11:04:38.137249: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x1d1a8490 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-21 11:04:38.137296: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2023-09-21 11:04:38.144955: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-21 11:04:38.212686: I tensor

85/85 [==============================] - ETA: 0s - loss: 0.0334 - mean_absolute_error: 0.1453
Epoch 1: val_loss improved from inf to 0.02413, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN+transformer_age%100(sigmoid)_loss(mse)_4fold/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_0.hdf5
85/85 [==============================] - 41s 321ms/step - loss: 0.0334 - mean_absolute_error: 0.1453 - val_loss: 0.0241 - val_mean_absolute_error: 0.1184
Epoch 2/100
85/85 [==============================] - ETA: 0s - loss: 0.0209 - mean_absolute_error: 0.1145
Epoch 2: val_loss improved from 0.02413 to 0.01873, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN+transformer_age%100(sigmoid)_loss(mse)_4fold/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_0.hdf5
85/85 [==============================] - 26s 302ms/step - loss: 0.0209 - mean_absolute_error: 0.1145 - val_loss: 0.0187 - val_mean_absol

Epoch 1/100
85/85 [==============================] - ETA: 0s - loss: 0.0327 - mean_absolute_error: 0.1440
Epoch 1: val_loss improved from inf to 0.02190, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN+transformer_age%100(sigmoid)_loss(mse)_4fold/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_1.hdf5
85/85 [==============================] - 39s 325ms/step - loss: 0.0327 - mean_absolute_error: 0.1440 - val_loss: 0.0219 - val_mean_absolute_error: 0.1130
Epoch 2/100
85/85 [==============================] - ETA: 0s - loss: 0.0213 - mean_absolute_error: 0.1156
Epoch 2: val_loss improved from 0.02190 to 0.01770, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN+transformer_age%100(sigmoid)_loss(mse)_4fold/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_1.hdf5
85/85 [==============================] - 26s 307ms/step - loss: 0.0213 - mean_absolute_error: 0.1156 - val_loss: 0.0177 - va

Epoch 1/100
85/85 [==============================] - ETA: 0s - loss: 0.0335 - mean_absolute_error: 0.1456
Epoch 1: val_loss improved from inf to 0.01961, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN+transformer_age%100(sigmoid)_loss(mse)_4fold/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_2.hdf5
85/85 [==============================] - 38s 318ms/step - loss: 0.0335 - mean_absolute_error: 0.1456 - val_loss: 0.0196 - val_mean_absolute_error: 0.1132
Epoch 2/100
85/85 [==============================] - ETA: 0s - loss: 0.0217 - mean_absolute_error: 0.1169
Epoch 2: val_loss did not improve from 0.01961
85/85 [==============================] - 26s 303ms/step - loss: 0.0217 - mean_absolute_error: 0.1169 - val_loss: 0.0201 - val_mean_absolute_error: 0.1088
Epoch 3/100
85/85 [==============================] - ETA: 0s - loss: 0.0179 - mean_absolute_error: 0.1055
Epoch 3: val_loss improved from 0.01961 to 0.01920, saving model to random

Epoch 1/100
85/85 [==============================] - ETA: 0s - loss: 0.0335 - mean_absolute_error: 0.1456
Epoch 1: val_loss improved from inf to 0.01979, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN+transformer_age%100(sigmoid)_loss(mse)_4fold/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_3.hdf5
85/85 [==============================] - 39s 321ms/step - loss: 0.0335 - mean_absolute_error: 0.1456 - val_loss: 0.0198 - val_mean_absolute_error: 0.1160
Epoch 2/100
85/85 [==============================] - ETA: 0s - loss: 0.0211 - mean_absolute_error: 0.1154
Epoch 2: val_loss improved from 0.01979 to 0.01804, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_female_train/CNN+transformer_age%100(sigmoid)_loss(mse)_4fold/batch128_cnn3_filt16_size5_pool2_do0.2_tra4_head8_kdim16_fnn128/weights_3.hdf5
85/85 [==============================] - 26s 308ms/step - loss: 0.0211 - mean_absolute_error: 0.1154 - val_loss: 0.0180 - va

## CNN-transformer + feats concat

### aug (12, 8, 6, 4)

In [ ]:
from keras.models import Model
from keras.layers import Layer, LayerNormalization, Dense, Dropout, Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Input, Concatenate, multiply, dot, MultiHeadAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

# learning scheduler
def step_decay(epoch):
    start = 1e-3
    drop = 0.1
    epochs_drop = 10
    lr = start * (drop ** np.floor((epoch)/epochs_drop))
    return lr

random.seed(98)
SRATE = 500  # in hz
SEGLEN = 10 * SRATE  # samples
#BATCH_SIZE = 256
MAX_CASES = 500
nfold = 4

hyperparameters = {
    "nfilt" : [16, 32, 64, 128],
    "stride": [2,4,5],
    'nhead' : [2, 4, 8],
    'kdim': [16, 64, 128, 256],
    "fnode" : [32, 64, 128],
    "clayer" : [2, 3, 4],
    "tlayer" : [3, 4, 5],
    "droprate" : [0.1, 0.2, 0.5],
    "filtsize" : [7, 9, 11, 15, 19],
    'poolsize' : [2, 4, 5],
    'batch_size': [64, 128, 256]
}
keys, values = zip(*hyperparameters.items())
permutations_dicts = it.product(*values)
permutations_dicts = list(permutations_dicts)
random.shuffle(permutations_dicts)
for nfilt, stride, nhead, kdim, fnode, clayer, tlayer, droprate, filtsize, poolsize, batch_size in permutations_dicts:
    test_start = time.time()

    tf.keras.backend.clear_session()
    
    rootdir = f'randomSearch/{hyper_path}/CNN+transformer+feats_4augs_age%{SCALE_Y}(sigmoid)_loss(mae)_{nfold}fold'
    odir_f = 'batch{}_cnn{}_filt{}_stride{}_size{}_pool{}_do{}'.format(batch_size, clayer, nfilt, stride, filtsize, poolsize, droprate)
    odir_f += '_tra{}_head{}_kdim{}_fnn{}'.format(tlayer, nhead, kdim, fnode)
    
    if not os.path.exists(rootdir):
        os.mkdir(rootdir)
    
    odir = rootdir+'/'+odir_f
    print("============================")
    print(odir)
    print("============================")

    # cnn-transformer
    # https://keras.io/examples/nlp/text_classification_with_transformer/
    #strategy = tf.distribute.MirroredStrategy()
    #with strategy.scope():
    with tf.device(f'/gpu:{GPU}'):
        out = inp = Input(shape=(x_train.shape[1], x_train.shape[2]), name='input_1')

        # out = Conv1D(filters=kdim, kernel_size=filtsize, padding='same')(out)

        # conv 여러층    
        for i in range(clayer):
            out = Conv1D(filters=nfilt, kernel_size=filtsize, strides=stride, padding='same', activation=None)(out)
            out = BatchNormalization()(out)
            out = Activation('relu')(out)
            out = MaxPooling1D(poolsize, padding='same')(out)
        out = Dense(kdim)(out)  # 마지막 차원이 nfilt 인데 kdim 으로 바꿔야 transformer block을 쌓을 수 있다.
        for i in range(tlayer):  # transformer
            attn_output = MultiHeadAttention(num_heads=nhead, key_dim=kdim, attention_axes=[1,])(out, out)
            attn_output = Dropout(droprate)(attn_output)
            out1 = LayerNormalization(epsilon=1e-6)(out + attn_output)  # sum and norm
            ffn_output = tf.keras.Sequential([Dense(fnode, activation="relu"), Dense(kdim)])(out1)
            out2 = Dropout(droprate)(ffn_output)
            out = LayerNormalization(epsilon=1e-6)(out1 + out2)  # sum and norm
        out = GlobalMaxPooling1D()(out)

        # concat with X_feature
        inp_feat = Input(shape=(feat_train.shape[1]), name="input_2")
        out = Concatenate(axis=1)([out, inp_feat])
        
        if droprate:
            out = Dropout(droprate)(out)
        out = Dense(fnode)(out)
        if droprate:
            out = Dropout(droprate)(out)
        out = Dense(1, activation='sigmoid')(out)

        if not os.path.exists(odir):
            os.mkdir(odir)

        #cache_path = odir + "/initial_weights.hdf5"
        model = Model(inputs=[inp, inp_feat], outputs=[out])
        lr_scheduler = LearningRateScheduler(step_decay, verbose=1)
        model.compile(loss='mae', optimizer=Adam(lr=lr_scheduler, weight_decay=None), metrics=['mean_squared_error'])
        model.save_weights(f'{odir}/initial_weights.hdf5')
    
    
    # 4-fold cv
    kfold = KFold(nfold)
    tprs, aucs, prs = [], [], []
    test_rmse, test_mae = [], []
    f1_scores, thvals = [], []
    mean_fpr = np.linspace(0,1,100)
    mean_recall = np.linspace(0,1,100)

    switch = 0
    caseids_train = np.unique(c_train)
    for fold, (c_cv_trains_mask, c_cv_test_mask) in enumerate(kfold.split(caseids_train)):
        c_cv_trains = caseids_train[c_cv_trains_mask]

        cv_train_mask = np.isin(c_train, c_cv_trains)
        cv_val_mask = ~cv_train_mask

        X_train = x_train[cv_train_mask]
        X_val = x_train[cv_val_mask]

        Y_train = y_train[cv_train_mask]
        Y_val = y_train[cv_val_mask]
        
        Feat_train = feat_train[cv_train_mask]
        Feat_val = feat_train[cv_val_mask]

        # model 학습
        try:
            # loading dataset
            with tf.device("/CPU:0"):
                train_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": X_train, "input_2": Feat_train}, Y_train)).shuffle(x_train.shape[0]).batch(batch_size)
                val_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": X_val, "input_2": Feat_val}, Y_val)).batch(batch_size)
                test_dataset = tf.data.Dataset.from_tensor_slices(({"input_1": x_test, "input_2": feat_test}, y_test)).batch(batch_size)
                
            weightcache = f"{odir}/weights_{fold}.hdf5"
            #model = multi_gpu_model(model, gpus=4)
            #model.compile(loss='mae', optimizer=Adam(lr=lr_scheduler, weight_decay=None), metrics=['mean_squared_error'])
            hist = model.fit(train_dataset, validation_data=val_dataset, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])

            model.load_weights(weightcache)
            y_pred = model.predict(test_dataset).flatten()

            # MAE 계산
            model_err = metrics.MeanAbsoluteError()
            model_err.update_state(y_test, y_pred)
            mae_val = model_err.result().numpy()
            test_mae.append(mae_val)



            print(f' ###{fold} fold : val mae {mae_val:.2f}###')
            tf.keras.backend.clear_session()
            model.load_weights(f'{odir}/initial_weights.hdf5')

        except Exception as e:
            print(e)
            switch = 1
            shutil.rmtree(odir)
            break
    ###
    if switch:
        switch = 0
        tf.keras.backend.clear_session()
        continue


    mean_mae = np.mean(test_mae)
    std_mae = np.std(test_mae)

    max_idx = test_mae.index(min(test_mae))


    print(f'mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{mean_mae*SCALE_Y:.2f}+-{std_mae*SCALE_Y:.2f}_max{max_idx}__{odir_f}')
    tf.keras.backend.clear_session()


randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer+feats_4augs_age%100(sigmoid)_loss(mae)_4fold/batch256_cnn4_filt16_stride4_size11_pool5_do0.2_tra5_head2_kdim256_fnn128


2023-09-22 03:23:52.113921: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 03:23:52.492070: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22324 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:82:00.0, compute capability: 8.6


Epoch 1/100


2023-09-22 03:24:13.377703: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2023-09-22 03:24:14.444938: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-22 03:24:14.521772: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-22 03:24:14.536896: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x16aed2db0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-22 03:24:14.536938: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2023-09-22 03:24:14.546468: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-22 03:24:14.617243: I tenso

157/157 [==============================] - ETA: 0s - loss: 0.3794 - mean_squared_error: 0.1695
Epoch 1: val_loss improved from inf to 0.38420, saving model to randomSearch/SRATE500_ECG-bandpass_z-norm_adult_male_train/CNN+transformer+feats_4augs_age%100(sigmoid)_loss(mae)_4fold/batch256_cnn4_filt16_stride4_size11_pool5_do0.2_tra5_head2_kdim256_fnn128/weights_0.hdf5
157/157 [==============================] - 30s 77ms/step - loss: 0.3794 - mean_squared_error: 0.1695 - val_loss: 0.3842 - val_mean_squared_error: 0.1728
Epoch 2/100
156/157 [============================>.] - ETA: 0s - loss: 0.3799 - mean_squared_error: 0.1698
Epoch 2: val_loss did not improve from 0.38420
157/157 [==============================] - 9s 56ms/step - loss: 0.3799 - mean_squared_error: 0.1698 - val_loss: 0.3842 - val_mean_squared_error: 0.1728
Epoch 3/100
156/157 [============================>.] - ETA: 0s - loss: 0.3798 - mean_squared_error: 0.1697
Epoch 3: val_loss did not improve from 0.38420
157/157 [==========